In [1]:
import pandas as pd
import numpy as np
import calendar
from time import time
from datetime import datetime
import re 
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import log_loss, accuracy_score, classification_report
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_train = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")
data_train_orig = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")

In [3]:
dept_list = sorted(list(data_train.DepartmentDescription.dropna().unique()))

In [4]:
weekdays = list(calendar.day_name)
dept_list_sum = dict.fromkeys(dept_list, np.sum)
weekday_dict = dict.fromkeys(weekdays, np.max)
feature_dict = {"TripType": np.max, 'NumItems': np.sum, 'Return': np.max}
feature_dict = {**feature_dict, **weekday_dict, **dept_list_sum}

In [5]:
def transform_data(data):
    dummies = pd.get_dummies(data.Weekday)
    data[dummies.columns] = dummies
    
    dummies = pd.get_dummies(data.DepartmentDescription)
    dummies = dummies.apply(lambda x: x*data["ScanCount"])
    data[dummies.columns] = dummies 

    data.loc[data.ScanCount < 0, 'Return'] = 1
    data.loc[data.Return != 1, 'Return'] = 0
    
    data = data.rename(columns={"ScanCount":"NumItems"})
    
    grouped = data.groupby("VisitNumber")
    grouped = grouped.aggregate(feature_dict)
    data = grouped[["TripType", "NumItems", "Return"] + weekdays + dept_list]

    return data

In [6]:
data_new = transform_data(data_train)

In [7]:
def add_category_counts(data):
    alist = []
    for array in np.asarray(data.loc[:, dept_list[0]:]):
        count = 0
        count = sum(x > 0 for x in array)
        alist.append(count)
    cat_counts = pd.DataFrame(alist)
    cat_counts = cat_counts.rename(columns={0:"CategoryCount"})
    cat_counts = cat_counts.set_index(data.index)
    data.insert(3, 'CategoryCounts', cat_counts)
    return data

In [8]:
data_new_cat = add_category_counts(data_new)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
X = data_new_cat.drop('TripType', axis=1)

trip_types = sorted(data_new_cat.TripType.unique())
trip_types_map = dict(zip(trip_types, np.arange(0, len(trip_types))))
y = data_new_cat.TripType.map(trip_types_map)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
np.random.seed(42)

In [13]:
def create_model_2hl(nodes_l1, nodes_l2, dropout_l1, dropout_l2):
    model = Sequential()
    model.add(Dense(output_dim=nodes_l1, activation="relu", input_dim=len(X.columns)))
    model.add(Dropout(dropout_l1))
    model.add(Dense(output_dim=nodes_l2, activation="relu"))
    model.add(Dropout(dropout_l2))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [14]:
model_2hl = KerasClassifier(build_fn=create_model_2hl, epochs=70, verbose=2)

In [15]:
nodes_l1 = np.arange(100, 150, 10)
nodes_l2 = np.arange(50, 100, 10)

dropout_l1 = np.arange(0.1, 0.4, 0.1)
dropout_l2 = np.arange(0.1, 0.4, 0.1)
dropout_l3 = np.arange(0.1, 0.4, 0.1)
batch_size = [500, 1000, 2000, 3000]

In [16]:
params2 = dict(nodes_l1=nodes_l1, nodes_l2=nodes_l2, dropout_l1=dropout_l1, dropout_l2=dropout_l2, batch_size=batch_size)

In [17]:
params2

{'nodes_l1': array([100, 110, 120, 130, 140]),
 'nodes_l2': array([50, 60, 70, 80, 90]),
 'dropout_l1': array([0.1, 0.2, 0.3, 0.4]),
 'dropout_l2': array([0.1, 0.2, 0.3, 0.4]),
 'batch_size': [500, 1000, 2000, 3000]}

In [18]:
scoring = ['neg_log_loss', 'accuracy']
cv = StratifiedKFold(n_splits=4)

In [19]:
RS2 = RandomizedSearchCV(estimator=model_2hl, param_distributions=params2, cv=cv, n_iter = 3, 
                         scoring=scoring, refit='accuracy', random_state=42, n_jobs=1, verbose=2)

In [20]:
RS2

RandomizedSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
                   error_score='raise-deprecating',
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001AD030DA390>,
                   iid='warn', n_iter=3, n_jobs=1,
                   param_distributions={'batch_size': [500, 1000, 2000, 3000],
                                        'dropout_l1': array([0.1, 0.2, 0.3, 0.4]),
                                        'dropout_l2': array([0.1, 0.2, 0.3, 0.4]),
                                        'nodes_l1': array([100, 110, 120, 130, 140]),
                                        'nodes_l2': array([50, 60, 70, 80, 90])},
                   pre_dispatch='2*n_jobs', random_state=42, refit='accuracy',
                   return_train_score=False,
                   scoring=['neg_log_loss', 'accuracy'], verbose=2)

In [21]:
start = datetime.now()
RS2.fit(X, y)
end = datetime.now()
str(end-start)

Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
W0904 19:51:32.072996 11924 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



[CV] nodes_l2=60, nodes_l1=100, dropout_l2=0.2, dropout_l1=0.4, batch_size=2000 


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=100)`
  This is separate from the ipykernel package so we can avoid doing imports until
W0904 19:51:32.280990 11924 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 19:51:32.336960 11924 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0904 19:51:32.388926 11924 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0904 19:51:32.4069

Epoch 1/70
 - 3s - loss: 3.2915 - acc: 0.1654
Epoch 2/70
 - 1s - loss: 2.5664 - acc: 0.3481
Epoch 3/70
 - 1s - loss: 2.0684 - acc: 0.4527
Epoch 4/70
 - 1s - loss: 1.7750 - acc: 0.5229
Epoch 5/70
 - 1s - loss: 1.5979 - acc: 0.5615
Epoch 6/70
 - 1s - loss: 1.4848 - acc: 0.5907
Epoch 7/70
 - 1s - loss: 1.4085 - acc: 0.6049
Epoch 8/70
 - 1s - loss: 1.3501 - acc: 0.6170
Epoch 9/70
 - 1s - loss: 1.3076 - acc: 0.6247
Epoch 10/70
 - 1s - loss: 1.2763 - acc: 0.6286
Epoch 11/70
 - 1s - loss: 1.2471 - acc: 0.6346
Epoch 12/70
 - 1s - loss: 1.2320 - acc: 0.6368
Epoch 13/70
 - 1s - loss: 1.2167 - acc: 0.6393
Epoch 14/70
 - 1s - loss: 1.1943 - acc: 0.6441
Epoch 15/70
 - 1s - loss: 1.1822 - acc: 0.6450
Epoch 16/70
 - 1s - loss: 1.1732 - acc: 0.6469
Epoch 17/70
 - 1s - loss: 1.1607 - acc: 0.6489
Epoch 18/70
 - 1s - loss: 1.1541 - acc: 0.6500
Epoch 19/70
 - 1s - loss: 1.1439 - acc: 0.6516
Epoch 20/70
 - 1s - loss: 1.1373 - acc: 0.6512
Epoch 21/70
 - 1s - loss: 1.1321 - acc: 0.6542
Epoch 22/70
 - 1s - lo

...

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=120)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=50)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/70
 - 2s - loss: 3.1984 - acc: 0.2108
Epoch 2/70
 - 1s - loss: 2.4004 - acc: 0.4138
Epoch 3/70
 - 1s - loss: 1.8908 - acc: 0.5050
Epoch 4/70
 - 1s - loss: 1.6193 - acc: 0.5606
Epoch 5/70
 - 1s - loss: 1.4617 - acc: 0.5948
Epoch 6/70
 - 1s - loss: 1.3616 - acc: 0.6134
Epoch 7/70
 - 1s - loss: 1.2997 - acc: 0.6268
Epoch 8/70
 - 1s - loss: 1.2551 - acc: 0.6344
Epoch 9/70
 - 1s - loss: 1.2173 - acc: 0.6416
Epoch 10/70
 - 1s - loss: 1.1949 - acc: 0.6457
Epoch 11/70
 - 1s - loss: 1.1719 - acc: 0.6496
Epoch 12/70
 - 1s - loss: 1.1501 - acc: 0.6538
Epoch 13/70
 - 1s - loss: 1.1362 - acc: 0.6553
Epoch 14/70
 - 1s - loss: 1.1248 - acc: 0.6561
Epoch 15/70
 - 1s - loss: 1.1087 - acc: 0.6605
Epoch 16/70
 - 1s - loss: 1.1011 - acc: 0.6608
Epoch 17/70
 - 1s - loss: 1.0907 - acc: 0.6623
Epoch 18/70
 - 1s - loss: 1.0817 - acc: 0.6636
Epoch 19/70
 - 1s - loss: 1.0744 - acc: 0.6666
Epoch 20/70
 - 1s - loss: 1.0700 - acc: 0.6645
Epoch 21/70
 - 1s - loss: 1.0608 - acc: 0.6682
Epoch 22/70
 - 1s - lo

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 21.1min finished
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=77, units=110)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=90)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/70
 - 3s - loss: 3.2696 - acc: 0.1893
Epoch 2/70
 - 2s - loss: 2.4317 - acc: 0.3825
Epoch 3/70
 - 2s - loss: 1.9288 - acc: 0.4930
Epoch 4/70
 - 2s - loss: 1.6440 - acc: 0.5566
Epoch 5/70
 - 2s - loss: 1.4786 - acc: 0.5908
Epoch 6/70
 - 2s - loss: 1.3744 - acc: 0.6110
Epoch 7/70
 - 2s - loss: 1.3037 - acc: 0.6263
Epoch 8/70
 - 2s - loss: 1.2550 - acc: 0.6317
Epoch 9/70
 - 2s - loss: 1.2178 - acc: 0.6395
Epoch 10/70
 - 2s - loss: 1.1944 - acc: 0.6441
Epoch 11/70
 - 2s - loss: 1.1701 - acc: 0.6487
Epoch 12/70
 - 2s - loss: 1.1516 - acc: 0.6513
Epoch 13/70
 - 2s - loss: 1.1363 - acc: 0.6545
Epoch 14/70
 - 2s - loss: 1.1225 - acc: 0.6557
Epoch 15/70
 - 2s - loss: 1.1115 - acc: 0.6565
Epoch 16/70
 - 2s - loss: 1.1024 - acc: 0.6581
Epoch 17/70
 - 2s - loss: 1.0933 - acc: 0.6625
Epoch 18/70
 - 2s - loss: 1.0865 - acc: 0.6617
Epoch 19/70
 - 2s - loss: 1.0766 - acc: 0.6639
Epoch 20/70
 - 2s - loss: 1.0700 - acc: 0.6664
Epoch 21/70
 - 2s - loss: 1.0630 - acc: 0.6658
Epoch 22/70
 - 2s - lo

'0:23:11.922320'

In [22]:
pd.DataFrame(RS2.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_nodes_l2', 'param_nodes_l1', 'param_dropout_l2',
       'param_dropout_l1', 'param_batch_size', 'params',
       'split0_test_neg_log_loss', 'split1_test_neg_log_loss',
       'split2_test_neg_log_loss', 'split3_test_neg_log_loss',
       'mean_test_neg_log_loss', 'std_test_neg_log_loss',
       'rank_test_neg_log_loss', 'split0_test_accuracy',
       'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy',
       'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy'],
      dtype='object')

In [23]:
RS2.best_params_

{'nodes_l2': 90,
 'nodes_l1': 110,
 'dropout_l2': 0.30000000000000004,
 'dropout_l1': 0.30000000000000004,
 'batch_size': 3000}

In [24]:
RS2.best_score_

0.701967096598867

In [27]:
cols = ['rank_test_neg_log_loss', 'mean_test_neg_log_loss', 'rank_test_accuracy', 'mean_test_accuracy', 
        'param_nodes_l1', 'param_nodes_l2', 'param_dropout_l1', 'param_dropout_l2','param_batch_size']

In [28]:
df = pd.DataFrame(RS2.cv_results_)
df[cols]

,rank_test_neg_log_loss,mean_test_neg_log_loss,rank_test_accuracy,mean_test_accuracy,param_nodes_l1,param_nodes_l2,param_dropout_l1,param_dropout_l2,param_batch_size
0,3,-0.900688,3,0.700985,100,60,0.4,0.2,2000
1,1,-0.888039,1,0.701967,110,90,0.3,0.3,3000
2,2,-0.888442,2,0.701779,120,50,0.1,0.3,2000


In [29]:
X.shape

(95674, 77)